# Module 7 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Unification

This is actually Part I of a two part assignment. In a later module, you'll implement a Forward Planner. In order to do that, however, you need to have a unifier. It is important to note that you *only* need to implement a unifier. Although the module talked about resolution, you do not need to implement anything like "standardizing apart". From the unifier's point of view, that should already have been done.

Unification is simply the *syntactic* balancing of expressions. There are only 3 kinds of expressions: constants, lists and (logic) variables. Constants and lists are only equal to each other if they're exactly the same thing or can be made to be the same thing by *binding* a value to a variable.

It really is that simple...expressions must be literally the same (identical) except if one or the other (or both) has a variable in that "spot".

## S-Expressions

With that out of the way, we need a language with which to express our constants, variables and predicates and that language will be based on s-expressions.

**constants** - There are two types of constants, values and predicates. Values should start with an uppercase letter. Fred is a constant value, so is Barney and Food. Predicates are named using lowercase letters. loves is a predicate and so is hates. This is only a convention. Secret: your code does not need to treat these two types of constants differently.

**variables** - these are named using lowercase letters but always start with a question mark. ?x is a variable and so is ?yum. This is not a convention.

**expressions (lists)** - these use the S-expression syntax a la LISP. (loves Fred Wilma) is an expression as is (friend-of Barney Fred) and (loves ?x ?y).

## Parsing

In [1]:
import tokenize
from io import StringIO

This uses the above libraries to build a Lisp structure based on atoms. It is adapted from [simple iterator parser](http://effbot.org/zone/simple-iterator-parser.htm) (deadlink). The first function is the `atom` function.

In [2]:
def atom( next, token):
    if token[ 1] == '(':
        out = []
        token = next()
        while token[ 1] != ')':
            out.append( atom( next, token))
            token = next()
            if token[ 1] == ' ':
                token = next()
        return out
    elif token[ 1] == '?':
        token = next()
        return "?" + token[ 1]
    else:
        return token[ 1]

The next function is the actual `parse` function:

In [3]:
def parse(exp):
    src = StringIO(exp).readline
    tokens = tokenize.generate_tokens(src)
    return atom(tokens.__next__, tokens.__next__())

**Note** there was a change between 2.7 and 3.0 that "hid" the next() function in the tokenizer.

From a Python perspective, we want to turn something like "(loves Fred ?x)" to ["loves" "Fred" "?x"] and then work with the second representation as a list of strings. The strings then have the syntactic meaning we gave them previously.

In [4]:
parse("Fred")

'Fred'

In [5]:
parse( "?x")

'?x'

In [6]:
parse( "(loves Fred ?x)")

['loves', 'Fred', '?x']

In [7]:
parse( "(father_of Barney (son_of Barney))")

['father_of', 'Barney', ['son_of', 'Barney']]

## Unifier

Now that that's out of the way, here is the imperative pseudocode for unification. This is a classic recursive program with a number of base cases. Students for some reason don't like it, try the algorithm in the book, can't get it to work and then come back to this pseudocode.

Work through the algorithm by hand with your Self-Check examples if you need to but I'd suggest sticking with this implementation. It does work.

Here is imperative pseudocode for the algorithm:

```
def unification( exp1, exp2):
    # base cases
    if exp1 and exp2 are constants or the empty list:
        if exp1 = exp2 then return {}
        else return FAIL
    if exp1 is a variable:
        if exp1 occurs in exp2 then return FAIL
        else return {exp1/exp2}
    if exp2 is a variable:
        if exp2 occurs in exp1 then return FAIL
        else return {exp2/exp1}

    # inductive step
    first1 = first element of exp1
    first2 = first element of exp2
    result1 = unification( first1, first2)
    if result1 = FAIL then return FAIL
    apply result1 to rest of exp1 and exp2
    result2 = unification( rest of exp1, rest of exp2)
    if result2 = FAIL then return FAIL
    return composition of result1 and result2
```

`unification` can return...

1. `None` (if unification completely fails)
2. `{}` (the empty substitution list) or 
3. a substitution list that has variables as keys and substituted values as values, like {"?x": "Fred"}. 

Note that the middle case sometimes confuses people..."Sam" unifying with "Sam" is not a failure so you return {} because there were no variables so there were no substitutions. You do not need to further resolve variables. If a variable resolves to an expression that contains a variable, you don't need to do the substition.

If you think of a typical database table, there is a column, row and value. This Tuple is a *relation* and in some uses of unification, the "thing" in the first spot..."love" above is called the relation. If you have a table of users with user_id, username and the value then the relation is:

`(login ?user_id ?username)`

*most* of the time, the relation name is specified. But it's not impossible for the relation name to be represented by a variable:

`(?relation 12345 "smooth_operator")`

Your code should handle this case (the pseudocode does handle this case so all  you have to do is not futz with it).

Our type system is very simple. We can get by with just a few boolean functions. The first tests to see if an expression is a variable.

In [8]:
def is_variable( exp):
    return isinstance( exp, str) and exp[ 0] == "?"

In [9]:
is_variable( "Fred")

False

In [10]:
is_variable( "?fred")

True

The second tests to see if an expression is a constant:

In [11]:
def is_constant( exp):
    return isinstance( exp, str) and not is_variable( exp)

In [12]:
is_constant( "Fred")

True

In [13]:
is_constant( "?fred")

False

In [14]:
is_constant( ["loves", "Fred", "?wife"])

False

It might also be useful to know that:

<code>
type( "a")
&lt;type 'str'>
type( "a") == str
True
type( "a") == list
False
type( ["a"]) == list
True
</code>


You need to write the `unification` function described above. It should work with two expressions of the type returned by `parse`. See `unify` for how it will be called. It should return the result of unification for the two expressions as detailed above and in the book. It does not have to make all the necessary substitions (for example, if ?y is bound to ?x and 1 is bound to ?y, ?x doesn't have to be replaced everywhere with 1. It's enough to return {"?x":"?y", "?y":1}. For an actual application, you would need to fix this!)

-----

## `apply_substitution` <a id="apply_substitution"></a>

**Description:**
This function applies a given substitution to an expression (which can be either a list or a string) and returns the updated expression. If the expression contains variables, the function replaces them with their corresponding values from the substitution. This is a crucial part of the unification algorithm as it ensures that the substitutions are fully applied. Without this step, variables like `?x` may not be substituted correctly, leading to partial or incorrect unifications (e.g., `?y/son Barney` instead of `?y/son ?x`).

**Parameters:**
- `substitution` (`Dict`): A dictionary containing variable-value pairs, where each variable (key) is mapped to its substitution (value).
- `expression` (`list` | `str`): The expression to which the substitution is applied. This can be a list or a string.

**Returns:**
- `expression` (`list` | `str`): The updated expression with the substitution applied. This will be a list if the original expression was a list, otherwise a string.

In [15]:
def apply_substitution(substitution, expression):
    if isinstance(expression, list):
        return [apply_substitution(substitution, sub_exp) for sub_exp in expression]
    elif is_variable(expression) and expression in substitution:
        return substitution[expression]
    return expression

In [16]:
assert(apply_substitution({'?x': 'Wilma'}, '?x') == 'Wilma') #Singular sub
assert(apply_substitution({'?x': 'Wilma', '?y': 'Barney'}, ['?x', '?y']) == ['Wilma', 'Barney']) #Multiple Subs
assert(apply_substitution({'?x': 'Wilma', '?y': 'Barney'}, ['?x', 'Barney', '?y']) == ['Wilma','Barney', 'Barney']) #Multiple subs with non substitution in the middle of it

## `unification_base_case` <a id="unification_base_case"></a>

**Description:**
This function handles the base case of the unification algorithm, performing simple checks to determine if two expressions can be unified (i.e., substituted with each other) or if unification fails. It is an essential step, as it prevents unification from proceeding with incompatible expressions. Without this step, the algorithm would either return incorrect results or be unable to unify properly.

**Parameters:**
- `exp1` (`list` | `str`): A part of the first expression in the unification process
- `exp2` (`list` | `str`): A part of the second expression in the unification process

**Returns:**
- `None` or `Dict`: Returns `None` if unification fails (i.e., the expressions cannot be substituted). Returns an empty dictionary `{}` if both expressions are identical constants, or a dictionary with the substitution if a variable is unified with another expression.

In [17]:
def unification_base_case(exp1, exp2):
    if is_constant(exp1) and is_constant(exp2):
        return {} if exp1 == exp2 else None
    if is_variable(exp1):
        return None if exp1 in exp2 else {exp1: exp2}
    if is_variable(exp2):
        return None if exp2 in exp1 else {exp2: exp1}
    return None

In [18]:
assert(unification_base_case('Wilma', 'Wilma') == {}) #Same constant
assert(unification_base_case('?x', 'Wilma') == {'?x': 'Wilma'}) #Simple substitution
assert(unification_base_case({'?x', 'Barney'}, {'Wilma', '?y'}) == None) #Cant do this as it is a list, lists just return None to move the code along to the base case

## `unification` <a id="unification"></a>

**Description:**
This function implements the recursive step of the unification algorithm. It first checks whether the base case of unification can be applied using the `unification_base_case` function. If the base case fails, the function proceeds recursively, decomposing the expressions and attempting to unify their subcomponents. This allows for unification of more complex expressions by gradually breaking them down.

**Parameters:**
- `list_expression1` (`list` | `str`): A part or the entirety of the first expression in the unification process
- `list_expression2` (`list` | `str`): A part or the entirety of the second expression in the unification process

**Returns:**
- `None` or `Dict`: Returns `None` if unification fails (i.e., the expressions cannot be unified). Returns a dictionary of variable-to-value substitutions if unification is successful.

In [19]:
def unification(list_expression1, list_expression2):
    base_result = unification_base_case(list_expression1, list_expression2) # Split this out so this function would be under 20 lines
    if base_result is not None:
        return base_result
    if isinstance(list_expression1, list) and isinstance(list_expression2, list):
        if not list_expression1 and not list_expression2:
            return {}
        if not list_expression1 or not list_expression2: 
            return None
        first1, first2 = list_expression1[0], list_expression2[0]
        result1 = unification(first1, first2)
        if result1 is None:
            return None
        rest1 = apply_substitution(result1, list_expression1[1:])
        rest2 = apply_substitution(result1, list_expression2[1:])
        result2 = unification(rest1, rest2)
        if result2 is None:
            return None
        result1.update(result2)
        return result1
    return None

In [20]:
assert(unification(['son', 'Barney', 'Barney'], ['daughter', 'Wilma', 'Pebbles']) == None) # Fail as Everything is constants that don't match
assert(unification(['father', '?x', '?x'], ['father', 'Fred', 'Barney']) == None) # Cant reassign ?x
assert(unification(['father', '?x'], ['father', 'Fred']) == {'?x': 'Fred'}) # Simple unification

In [21]:
def list_check(parsed_expression):
    if isinstance(parsed_expression, list):
        return parsed_expression
    return [parsed_expression]

The `unification` pseudocode only takes lists so we have to make sure that we only pass a list.
However, this has the side effect of making "foo" unify with ["foo"], at the start.
That's ok.

In [22]:
def unify( s_expression1, s_expression2):
    list_expression1 = list_check(parse(s_expression1))
    list_expression2 = list_check(parse(s_expression2))
    return unification( list_expression1, list_expression2)

**Note** If you see the error,

```
tokenize.TokenError: ('EOF in multi-line statement', (2, 0))
```
You most likely have unbalanced parentheses in your s-expression.

## Test Cases

Use the expressions from the Self Check as your test cases...

In [23]:
self_check_test_cases = [
    ['(son Barney Barney)', '(daughter Wilma Pebbles)', None],
    ['Pebbles', 'Pebbles', {}],
    ['(quarry_worker Fred)', '(quarry_worker ?x)', {'?x': 'Fred'}],
    ['(son Barney ?x)', '(son ?y Bam_Bam)', {'?y': 'Barney', '?x': 'Bam_Bam'}],
    ['(married ?x ?y)', '(married Barney Wilma)', {'?x': 'Barney', '?y': 'Wilma'}],
    ['(son Barney ?x)', '(son ?y (son Barney))', {'?y': 'Barney', '?x': ['son', 'Barney']}],
    ['(son Barney ?x)', '(son ?y (son ?y))', {'?y': 'Barney', '?x': ['son', 'Barney']}],
    ['(son Barney Bam_Bam)', '(son ?y (son Barney))', None],
    ['(loves Fred Fred)', '(loves ?x ?x)', {'?x': 'Fred'}],
    ['(future George Fred)', '(future ?y ?y)', None]
]
for case in self_check_test_cases:
    exp1, exp2, expected = case
    actual = unify(exp1, exp2)
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

actual = None
expected = None


actual = {}
expected = {}


actual = {'?x': 'Fred'}
expected = {'?x': 'Fred'}


actual = {'?y': 'Barney', '?x': 'Bam_Bam'}
expected = {'?y': 'Barney', '?x': 'Bam_Bam'}


actual = {'?x': 'Barney', '?y': 'Wilma'}
expected = {'?x': 'Barney', '?y': 'Wilma'}


actual = {'?y': 'Barney', '?x': ['son', 'Barney']}
expected = {'?y': 'Barney', '?x': ['son', 'Barney']}


actual = {'?y': 'Barney', '?x': ['son', 'Barney']}
expected = {'?y': 'Barney', '?x': ['son', 'Barney']}


actual = None
expected = None


actual = {'?x': 'Fred'}
expected = {'?x': 'Fred'}


actual = None
expected = None




Now add at least **five (5)** additional test cases of your own making, explaining exactly what you are testing. They should not be testing the same things as the self check test cases above.

In [24]:
new_test_cases = [
    ['(son Barney Barney)', '(daughter Wilma Pebbles)', None, "non-equal constants"],
    ['(loves ?x Wilma)', '(loves Fred Wilma)', {'?x': 'Fred'}, "simple variable matching"],
    ['(father ?x ?y)', '(father Fred Barney)', {'?x': 'Fred', '?y': 'Barney'}, "multiple variables"],
    ['(loves Fred ?x)', '(loves Fred Pebbles)', {'?x': 'Pebbles'}, "variable matches constant"],
    ['(knows ?x ?y)', '(knows ?y ?x)', None, "non-unifiable due to recursion"],
    ['(loves ?x ?x)', '(loves Fred Wilma)', None, "same variable but different constants"]

]
for case in new_test_cases:
    exp1, exp2, expected, message = case
    actual = unify(exp1, exp2)
    print(f"Testing {message}...")
    print(f"actual = {actual}")
    print(f"expected = {expected}")
    print("\n")
    assert expected == actual

Testing non-equal constants...
actual = None
expected = None


Testing simple variable matching...
actual = {'?x': 'Fred'}
expected = {'?x': 'Fred'}


Testing multiple variables...
actual = {'?x': 'Fred', '?y': 'Barney'}
expected = {'?x': 'Fred', '?y': 'Barney'}


Testing variable matches constant...
actual = {'?x': 'Pebbles'}
expected = {'?x': 'Pebbles'}


Testing non-unifiable due to recursion...
actual = None
expected = None


Testing same variable but different constants...
actual = None
expected = None




## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.

```
(blah foo ?x)
(blah foo ?x)
```